# Import necessary libraries

In [7]:
from litellm import completion
import re
import os
import time

# Define functions

In [8]:
def get_ai_completion(text):
    response = completion(
        model="ollama/mistral:7b-instruct", 
        messages=[{ "content": text,"role": "user"}], 
        api_base="http://localhost:11434",
        stream=False
    )

    return response.choices[0].message.content  

In [9]:
def fetch_sections_from_outline(outline):
    lines = outline.strip().split("\n")
    sections = []

    # Initialize an empty string to hold the current section
    current_section = ""

    for line in lines:
        # Ignore empty lines
        if not line.strip():
            continue
        # If the line starts with a roman numeral, it's a new section
        if re.match(r"^[IVXLCDM]+\.", line):
            # If there's a current section, add it to the sections list
            if current_section:
                sections.append(current_section)
            # Start a new section
            current_section = line.strip()
        # If the line starts with a letter and a period, it's a subsection
        elif re.match(r"^\s+[A-Z]\.", line):
            # Add the subsection to the current section
            current_section += "\n" + line.strip()

    # Add the last section to the sections list
    if current_section:
        sections.append(current_section)

    return sections

In [10]:
def save_text_as_markdown(text, topic):
    # Remove any leading or trailing quotes from topic if any
    topic = topic.strip().strip('"').strip("'")
    # Remove trailing period if any
    if topic.endswith('.'):
        topic = topic[:-1]
    folder_path = "blogs"
    os.makedirs(folder_path, exist_ok=True)
    file_path = os.path.join(folder_path, f"{topic}.md")
    with open(file_path, "w") as file:
        file.write(text)

In [11]:
blog_count = 20
topic = "Next.js"

In [12]:
topics = []

for i in range(blog_count):
    # Log time to generate each blog
    start_time = time.time()
    topics_str = ", ".join(topics)
    suggested_topic = get_ai_completion(f"Suggest a topic for a blog post about {topic} in one line. Don't include topics that are similar to these topics: {topics_str}. Suggest only one topic. Don't write a paragraph. Just one line. Don't mention anything else.")
    suggested_topic = suggested_topic.strip().strip('"').strip("'")

    print(f"Suggested Blog Topic {i+1}: {suggested_topic}")
    topics.append(suggested_topic)
    print("\n* Generating outline *")
    outline = get_ai_completion(f"Write an outline for a blog post about {suggested_topic}. Don't write a paragraph. Don't mention anything else.")
    sections = fetch_sections_from_outline(outline)

    for index, section in enumerate(sections):
        print(f"\nSection {index+1}:")
        print("----------")
        print(section)
    
    blog = f"# {suggested_topic}\n\n"

    print("\n* Generating content for each sections *")
    for index, section in enumerate(sections):
        print(f"* Section {index+1}: done")
        content = get_ai_completion(f"You are an experience content writer. You are capable of writing captivating, highly informative, seo optimized blogs. I'm going to give you a topic and each section, and you are going to write the content for that section in markdown format. Write a paragraph about this section: {section} on the topic: {suggested_topic}. Just write a paragraph. Don't mention anything else.")
        blog += f"\n\n{content}\n\n"

    print("\n* Formatting the blog in proper markdown format *")
    formatted_blog = get_ai_completion(f"Enhance the formatting of this markdown blog. Fix indentations, headings, and styles. Don't modify the actual content. Just fix the formatting. The blog post is about {suggested_topic}. \n\nBlog: {blog}")
    end_time = time.time()
    elapsed_time = end_time - start_time  # calculate the difference
    elapsed_time_in_minutes = elapsed_time / 60  # convert to minutes
    print("* done")
    print("------------------------")
    print(f"Time taken to generate the blog: {elapsed_time_in_minutes:.2f} minutes.")
    print("====================================\n\n\n")
    save_text_as_markdown(formatted_blog, suggested_topic)


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.



AttributeError: 'NoneType' object has no attribute 'request'